## 1. Load data from file

In [0]:
import numpy as np
import torch

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

DATA_PATH = './mnist.csv'
SIZE_ROW = 28
SIZE_COL = 28

# load data from file
data = torch.from_numpy(
    np.genfromtxt(DATA_PATH, delimiter=',')
)

# separate pixel values from labels
X_raw = data[:, 1:]
Y_raw = data[:, 0]

## 2. Data preprocessing

In [0]:
# normalize X data
X_data = (X_raw - torch.min(X_raw)) / (torch.max(X_raw) - torch.min(X_raw))

# one-hot encoding Y data
Y_data = Y_raw.to(torch.long)
Y_data = torch.eye(torch.unique(Y_data).shape[0])[Y_data]

# train data
X_train = X_data[:1000].to(device)
Y_train = Y_data[:1000].to(device)

# test data
X_test = X_data[1000:].to(device)
Y_test = Y_data[1000:].to(device)